In [41]:
import os
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import linregress
from scipy.stats import t
import sympy as sp

def sqrt_sumsq(arr):
    place_holder = []
    for i in arr:
        place_holder.append(i**2)
    return np.sqrt(np.sum(place_holder))

def Precision_Uncertainty(arr):
    
    #Precision Uncertainty Calculation Initialization
    Ns = len(arr)
    dof = Ns - 1  # degrees of freedom for mean
    t_crit = t.ppf(0.975, dof) #0.975 for two-tails student-t
    
    Prec_Unc = t_crit * np.nanstd(arr, ddof=1) / np.sqrt(Ns)
    return Prec_Unc

def Bias_Uncertainty(Acc, Res):
    
    Bias_Unc = sqrt_sumsq([Acc, Res/2])
    
    return Bias_Unc

def UPE_dPt(
    dPs_val, U_dPs_val,
    mass_val, U_mass_val,
    vol_val,  U_vol_val,
    dh_val,   U_dh_val,
    m_dot_val, U_mdot_val,
    A1_val, A2_val, U_A1_val, U_A2_val,
    g_val=9.807
):
    # 1) Symbolic setup with mass and volume
    dPs, mass, vol, g, dh, m_dot, A1, A2 = sp.symbols('dPs mass vol g dh m_dot A1 A2', real=True)
    U_dPs, U_mass, U_vol, U_dh, U_mdot, U_A1, U_A2 = sp.symbols('U_dPs U_mass U_vol U_dh U_mdot U_A1 U_A2', real=True, positive=True)
    
    # 2) Define rho = mass/vol and the formula for dPt
    rho = mass/vol
    dPt = (
        dPs
        + rho * g * dh
        + sp.Rational(1,2) * rho * ((m_dot/(rho*A2))**2 - (m_dot/(rho*A1))**2)
    )
    
    # 3) Compute symbolic partial derivatives w.r.t variables
    vars_sym = (dPs, mass, vol, dh, m_dot, A1, A2)
    partials = [sp.diff(dPt, var) for var in vars_sym]
    
    # 4) Build combined uncertainty U_dPt
    U_vars = (U_dPs, U_mass, U_vol, U_dh, U_mdot, U_A1, U_A2)
    U_dPt = sp.sqrt(sum((partials[i] * U_vars[i])**2 for i in range(len(vars_sym))))
    U_dPt_simplified = sp.simplify(U_dPt)
    
    # 5) Lambdify for numeric evaluation
    f_dPt      = sp.lambdify((dPs, mass, vol, g, dh, m_dot, A1, A2), dPt, 'numpy')
    f_partials = [sp.lambdify((dPs, mass, vol, g, dh, m_dot, A1, A2), pd, 'numpy') for pd in partials]

    # nominal result
    dPt_val = f_dPt(dPs_val, mass_val, vol_val, g_val, dh_val, m_dot_val, A1_val, A2_val)
    
    # compute gradients
    grads = np.array([f(dPs_val, mass_val, vol_val, g_val, dh_val, m_dot_val, A1_val, A2_val)
                      for f in f_partials], dtype=float)
    
    # uncertainties array
    U_vals = np.array([U_dPs_val, U_mass_val, U_vol_val, U_dh_val, U_mdot_val, U_A1_val, U_A2_val], dtype=float)
    
    # RSS propagation
    U_dPt_val = np.sqrt(np.sum((grads * U_vals)**2))
    return dPt_val, U_dPt_val, grads


def process_file(file_path):
    font_legend = {'family': 'TH Sarabun New',
                   'weight': 'bold',
                   'style': 'normal',
                   'size': 18}
    font_axis = {'family': 'TH Sarabun New',
                 'weight': 'bold',
                 'style': 'normal',
                 'size': 20}

    base_name  = os.path.splitext(os.path.basename(file_path))[0]
    parent_dir = os.path.dirname(file_path)
    output_dir = os.path.join(parent_dir, f"{base_name} plots")
    os.makedirs(output_dir, exist_ok=True)

    # Read CSV, skip first two summary rows, use third row as header
    df = pd.read_csv(file_path, delimiter=',', skiprows=2, header=0)
    x  = df['Run Time [s]']

    # 1) Mass Flow Rate vs Run Time (full) — raw data only, no curve fit
    plt.figure()
    plt.plot(x, df['Weight [Kg]'], label='Data')
    plt.xlabel('Run Time [s]', fontdict=font_axis)
    plt.ylabel('Weight [Kg]', fontdict=font_axis)
    plt.title('Mass Flow Rate vs Run Time', fontdict=font_axis)
    plt.legend(prop=font_legend)
    plt.ylim(bottom=0)
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, f"{base_name}_Mass_Flow_Rate_full_plot.png"))
    plt.close()

    # ... (Differential Pressure, Rotational Speed, Torque full-plot blocks unchanged) ...

    # --- FILTERED PLOTS (20 ≤ Weight ≤ 30 Kg) ---
    filtered = df[(df['Weight [Kg]'] >= 20) & (df['Weight [Kg]'] <= 30)]
    if not filtered.empty:
        
        frt = filtered['Run Time [s]'].iloc[-1] - filtered['Run Time [s]'].iloc[0]

        # Mass Flow Rate (filtered fit) + capture its standard error
        xf, yf = filtered['Run Time [s]'], filtered['Weight [Kg]']
        slope_f, intercept_f, r_f, _, stderr_f = linregress(xf, yf)
        # convert 1σ stderr into 95% CI half-width
        dof    = len(xf) - 2
        t_crit = t.ppf(0.975, dof)
        stderr_f *= t_crit
        r2_f = r_f**2
        max_mass = max(yf)
        min_mass = min(yf)
        plt.figure()
        plt.plot(xf, yf, label='Filtered')
        plt.plot(xf, slope_f*xf + intercept_f,
                 label=f'Fit: y={slope_f:.4f}x+{intercept_f:.4f}\nR²={r2_f:.4f}')
        plt.xlabel('Run Time [s]', fontdict=font_axis)
        plt.ylabel('Weight [Kg]', fontdict=font_axis)
        plt.title('Mass Flow Rate vs Run Time (Filtered)', fontdict=font_axis)
        plt.text(0.05, 0.95, f'Filtered Run Time: {frt:.2f} s',
                 transform=plt.gca().transAxes, verticalalignment='top', fontdict=font_axis)
        plt.legend(prop=font_legend); plt.ylim(bottom=0)
        plt.tight_layout()
        plt.savefig(os.path.join(output_dir, f"{base_name}_Mass_Flow_Rate_filtered_plot.png"))
        plt.close()

        # Differential Pressure stats + plot
        dp     = filtered['Differential Pressure [Bar]']
        avg_dp = dp.mean()
        std_dp = t_crit * dp.std(ddof=1)
        plt.figure()
        plt.plot(xf, dp)
        plt.xlabel('Run Time [s]', fontdict=font_axis)
        plt.ylabel('Differential Pressure [Bar]', fontdict=font_axis)
        plt.title('Differential Pressure vs Run Time (Filtered)', fontdict=font_axis)
        plt.text(0.05, 0.95,
                 f'Filtered Run Time: {frt:.2f} s\n'
                 f'Avg: {avg_dp:.4f} Bar\n'
                 f'SD:  {std_dp:.4f}',
                 transform=plt.gca().transAxes, verticalalignment='top', fontdict=font_axis)
        plt.ylim([0,3])
        plt.tight_layout()
        plt.savefig(os.path.join(output_dir, f"{base_name}_Differential_Pressure_filtered_plot.png"))
        plt.close()

        # Rotational Speed stats + plot
        rs     = filtered['Rotational Speed [RPM]']
        avg_rs = rs.mean()
        std_rs = t_crit * rs.std(ddof=1)
        plt.figure()
        plt.plot(xf, rs)
        plt.xlabel('Run Time [s]', fontdict=font_axis)
        plt.ylabel('Rotational Speed [RPM]', fontdict=font_axis)
        plt.title('Rotational Speed vs Run Time (Filtered)', fontdict=font_axis)
        plt.text(0.05, 0.95,
                 f'Filtered Run Time: {frt:.2f} s\n'
                 f'Avg: {avg_rs:.4f} RPM\n'
                 f'SD:  {std_rs:.4f}',
                 transform=plt.gca().transAxes, verticalalignment='top', fontdict=font_axis)
        plt.ylim([0,8500])
        plt.tight_layout()
        plt.savefig(os.path.join(output_dir, f"{base_name}_Rotational_Speed_filtered_plot.png"))
        plt.close()

        # Torque stats + plot
        tq     = filtered['Torque [N-m]']
        avg_tq = tq.mean()
        std_tq = t_crit * tq.std(ddof=1)
        plt.figure()
        plt.plot(xf, tq)
        plt.xlabel('Run Time [s]', fontdict=font_axis)
        plt.ylabel('Torque [N-m]', fontdict=font_axis)
        plt.title('Torque vs Run Time (Filtered)', fontdict=font_axis)
        plt.text(0.05, 0.95,
                 f'Filtered Run Time: {frt:.2f} s\n'
                 f'Avg: {avg_tq:.4f} N-m\n'
                 f'SD:  {std_tq:.4f}',
                 transform=plt.gca().transAxes, verticalalignment='top', fontdict=font_axis)
        plt.ylim([0,0.5])
        plt.tight_layout()
        plt.savefig(os.path.join(output_dir, f"{base_name}_Torque_filtered_plot.png"))
        plt.close()
        
    else:
        slope_f = stderr_f = np.nan
        avg_dp = avg_rs = avg_tq = std_dp = std_rs = std_tq = np.nan
        print(f"{base_name}: no data in weight range 20–30 Kg")

    # ---- RETURN original metrics PLUS S.E. of slope ----
    return {
        'file':                base_name,
        'mass_flow_rate':      slope_f,
        'se_mass_flow_rate':   stderr_f,     # added standard error
        'mean_dp':             avg_dp,
        'mean_rs':             avg_rs,
        'mean_torque':         avg_tq,
        'std_dp':              std_dp,
        'std_rs':              std_rs,
        'std_tq':              std_tq,
        'run_time':            frt,
        'max_mass':            max_mass,
        'min_mass':            min_mass
    }


def main(csv_dir):
    files = glob.glob(os.path.join(csv_dir, '*.csv'))
    if not files:
        print(f"No CSV files found in {csv_dir}")
        return

    summaries = [process_file(f) for f in files]

    # ARRAYS of the PER-FILE time-averaged MEANS (unchanged)
    mfr_arr = np.array([s['mass_flow_rate'] for s in summaries])
    dp_arr  = np.array([s['mean_dp']        for s in summaries])
    rs_arr  = np.array([s['mean_rs']        for s in summaries])
    tq_arr  = np.array([s['mean_torque']    for s in summaries])
    run_time_arr = np.array([s['run_time']    for s in summaries])
    max_mass_arr = np.array([s['max_mass']    for s in summaries])
    min_mass_arr = np.array([s['min_mass']    for s in summaries])
    
    # ARRAYS of the PER-FILE SDs (unchanged)
    sd_dp_arr = np.array([s['std_dp'] for s in summaries])
    sd_rs_arr = np.array([s['std_rs'] for s in summaries])
    sd_tq_arr = np.array([s['std_tq'] for s in summaries])

    # ARRAY of the PER-FILE S.E.
    se_mfr_arr = np.array([s['se_mass_flow_rate'] for s in summaries])

    # GRAND MEANS & SDs of the MEANS (unchanged)
    grand_means = {
        'mfr_mean':  np.nanmean(mfr_arr), 'mfr_sd':  np.nanstd(mfr_arr, ddof=1),
        'dp_mean':   np.nanmean(dp_arr),  'dp_sd':   np.nanstd(dp_arr,  ddof=1),
        'rs_mean':   np.nanmean(rs_arr),  'rs_sd':   np.nanstd(rs_arr,  ddof=1),
        'tq_mean':   np.nanmean(tq_arr),  'tq_sd':   np.nanstd(tq_arr,  ddof=1),
        'run_time_mean':   np.nanmean(run_time_arr),  'run_time_sd':   np.nanstd(run_time_arr,  ddof=1),
        'max_mass_mean':   np.nanmean(max_mass_arr),  'max_mass_sd':   np.nanstd(max_mass_arr,  ddof=1),
        'min_mass_mean':   np.nanmean(min_mass_arr),  'min_mass_sd':   np.nanstd(min_mass_arr,  ddof=1),
    } #Single Number

    # ARITHMETIC AVERAGE OF THE SDs (unchanged)
    avg_sd = {
        'dp_bar': np.nanmean(sd_dp_arr),
        'rs_bar': np.nanmean(sd_rs_arr),
        'tq_bar': np.nanmean(sd_tq_arr),
    }

    # AVERAGE STANDARD ERROR OF MASS FLOW RATE (added)
    #avg_se = np.nanmean(se_mfr_arr)
    
    grand_pu = {
        'mfr_pu':  Precision_Uncertainty(mfr_arr),
        'dp_pu':   Precision_Uncertainty(dp_arr),
        'rs_pu':   Precision_Uncertainty(rs_arr),
        'tq_pu':   Precision_Uncertainty(tq_arr)
    } #Single Number
    
    #Bias Uncertainty Code Area
    Sensor_Accuracy_Dict = {
        'mass_acc': 0.08426150, 'mass_res': 10 / 2**16 * 5, #kg
        'time_acc': 50e-6 * 0.05, 'time_res': 50e-9, #sec
        'dp_acc': 0.0075, 'dp_res': 10 / 2**16 * 0.3, #bar, res = 10V / 16-bit * 0.3 Bar/V, acc = 0.25%FS
        'tq_acc': 0.001, 'tq_res': 10 / 2**16 * 0.1, #N-m, res = 10V / 16-bit * 0.1 N-m/V
        'rs_acc': 0, 'rs_res': 1 #RPM   
    } #Single Number
    mass_bu = Bias_Uncertainty(Sensor_Accuracy_Dict['mass_acc'], Sensor_Accuracy_Dict['mass_res'])
    time_bu = Bias_Uncertainty(Sensor_Accuracy_Dict['time_acc'], Sensor_Accuracy_Dict['time_res'])
    dp_bu = Bias_Uncertainty(Sensor_Accuracy_Dict['dp_acc'], Sensor_Accuracy_Dict['dp_res'])
    rs_bu = Bias_Uncertainty(Sensor_Accuracy_Dict['rs_acc'], Sensor_Accuracy_Dict['rs_res'])
    tq_bu = Bias_Uncertainty(Sensor_Accuracy_Dict['tq_acc'], Sensor_Accuracy_Dict['tq_res'])
    
    
    
    #Derived Quantities Calculation : Total Pressure Rise | Shaft Power | Hydraulic Power | Efficiency
    
    #Individual values
    
    #Calculating mfr uncertainty
    mfr_BUF_arr = []
    mfr_bu = []
    for i in range(len(mfr_arr)):
        mfr_Bias_UPE = sqrt_sumsq([ mass_bu/run_time_arr[i] , mass_bu/run_time_arr[i] , (min_mass_arr[i] - max_mass_arr[i]) / run_time_arr[i]**2 * time_bu ])
        
        mfr_BUF = sqrt_sumsq([ mfr_Bias_UPE, se_mfr_arr[i]])
        mfr_bu.append(mfr_Bias_UPE)
        mfr_BUF_arr.append(mfr_BUF)
    
    mfr_BUF_mean = np.nanmean(mfr_BUF_arr)
    
    
    #Final Sum Uncertainty
    mfr_uncertainty = sqrt_sumsq([grand_pu['mfr_pu'], mfr_BUF_mean])
    dp_uncertainty = sqrt_sumsq([dp_bu, grand_pu['dp_pu'], avg_sd['dp_bar']])
    rs_uncertainty = sqrt_sumsq([rs_bu, grand_pu['rs_pu'], avg_sd['rs_bar']])
    tq_uncertainty = sqrt_sumsq([tq_bu, grand_pu['tq_pu'], avg_sd['tq_bar']])
    
    
    dPt_arr = []
    dPt_BUF_arr = []
    #Denote Bias Uncertainty + Fluctuation = BUF
    
    vernier_acc = 0.03/1000 #0.03 mm
    vernier_res = 0.02/1000 #0.02 mm
    
    #Calculating dPt value and uncertainty
    for i in range(len(dp_arr)):
        dPs, dPs_BUF = dp_arr[i]*100000, (sqrt_sumsq([sd_dp_arr[i], dp_bu]))*100000
        mass = [1.1762, 1.1771, 1.1763, 1.1781, 1.1759]
        mass_mean = np.nanmean(mass) 
        U_mass = sqrt_sumsq([Bias_Uncertainty(0.0001, 0.0001) , Precision_Uncertainty(mass)]) # kg, uncertainty for Density Calculation
        
        vol,  U_vol  = 0.001, Bias_Uncertainty(0.4/1000000 , 0)  # m^3, uncertainty for Density Calculation ±0.3 mL for 1000 mL Volumetric Flask
        
        dh = [0.09304, 0.09288, 0.09296, 0.09300, 0.09306] #m
        dh_mean = np.nanmean(dh)
        U_dh = sqrt_sumsq([Bias_Uncertainty(0.03/1000, 0.02/1000) , Precision_Uncertainty(dh) ])
        
        m_dot, U_mdot = mfr_arr[i], mfr_BUF_arr[i] # kg/s
        
        D1 = [18.08 / 1000, 18.12 / 1000, 18.08 / 1000, 18.14 / 1000, 18.12 / 1000]
        D1_mean = np.nanmean(D1)
        D1_uncertainty = sqrt_sumsq([Bias_Uncertainty(vernier_acc, vernier_res), Precision_Uncertainty(D1)])
        
        D2 = [13.36 / 1000, 13.42 / 1000, 13.38 / 1000, 13.30 / 1000, 13.36 / 1000]
        D2_mean = np.nanmean(D2)
        D2_uncertainty = sqrt_sumsq([Bias_Uncertainty(vernier_acc, vernier_res), Precision_Uncertainty(D2)])
        
        A1 = [(np.pi / 4 * i**2) for i in D1]
        A2 = [(np.pi / 4 * i**2) for i in D2]
        A1_mean = np.nanmean(A1)
        A2_mean = np.nanmean(A2)
        A1_uncertainty = sqrt_sumsq([(np.pi / 2 * D1_mean * D1_uncertainty),Precision_Uncertainty(A1)])
        A2_uncertainty = sqrt_sumsq([(np.pi / 2 * D2_mean * D2_uncertainty),Precision_Uncertainty(A2)])
        
        dPt, dPt_uncertainty , _ = UPE_dPt(
        dPs, dPs_BUF, mass_mean, U_mass, vol, U_vol,
        dh_mean, U_dh, m_dot, U_mdot,
        A1_mean, A2_mean, A1_uncertainty, A2_uncertainty)
            
        dPt_arr.append(dPt)
        dPt_BUF_arr.append(dPt_uncertainty)
          
    dPt_mean = np.nanmean(dPt_arr) #Representative Value
    dPt_BUF_mean = np.nanmean(dPt_BUF_arr) #Used as Representative Bias Uncertainty of dPt
    
    dPt_mean = dPt_mean/100000 #Convert Pa back to Bar
    dPt_uncertainty = sqrt_sumsq([dPt_BUF_mean, Precision_Uncertainty(dPt_arr)])/100000 #Combined BUF and PU
    #Finished dPt Calculation
    
    #Calculating shaft power
    shaft_power_arr = tq_arr * rs_arr * 2*np.pi/60
    
    shaft_power_BUF_arr = []
    for i in range(len(shaft_power_arr)):
        tq_BUF = sqrt_sumsq([ tq_bu ,  sd_tq_arr[i] ]) #Torque BUF of an individual sample
        rs_BUF = sqrt_sumsq([ rs_bu ,  sd_rs_arr[i] ]) #RPM BUF of an individual sample
        shaft_power_UPE = sqrt_sumsq([ rs_arr[i] * 2 * np.pi / 60 * tq_BUF, tq_arr[i] * 2 * np.pi / 60 * rs_BUF])
        shaft_power_BUF_arr.append(shaft_power_UPE)
    
    shaft_power_BUF_mean = np.nanmean(shaft_power_BUF_arr)
    shaft_power_uncertainty = sqrt_sumsq([ shaft_power_BUF_mean, Precision_Uncertainty(shaft_power_arr) ])
    shaft_power_mean = np.nanmean(shaft_power_arr)
    
    mass = [1.1762, 1.1771, 1.1763, 1.1781, 1.1759]
    mass_mean = np.nanmean(mass) 
    U_mass = sqrt_sumsq([Bias_Uncertainty(0.0001, 0.0001) , Precision_Uncertainty(mass)]) # kg, uncertainty for Density Calculation
    vol,  U_vol  = 0.001, Bias_Uncertainty(0.4/1000000 , 0)  # m^3, uncertainty for Density Calculation ±0.3 mL for 1000 mL Volumetric Flask
    # Density Uncertainty Propagation Equation
    density = mass_mean / vol
    density_uncertainty = sqrt_sumsq([U_mass / vol, -1 * mass_mean/vol**2 * U_vol])
    #Finished shaft power calculation
    
    #Calculating hydraulic power
    hydraulic_power_arr = dPt_arr * (mfr_arr / density)
    
    hydraulic_power_BUF_arr = []
    for i in range(len(hydraulic_power_arr)):
        #dPt_arr + dPt_BUF_arr
        #density + density_uncertainty
        #mfr_arr + se_mfr_arr
        hydraulic_power_UPE = sqrt_sumsq([ mfr_arr[i] / density * dPt_BUF_arr[i],   
                                           dPt_arr[i] / density * mfr_BUF_arr[i], 
                                           -1 * dPt_arr[i] *  mfr_arr[i] / density**2 * density_uncertainty])
        hydraulic_power_BUF_arr.append(hydraulic_power_UPE)
    
    hydraulic_power_BUF_mean = np.nanmean(hydraulic_power_BUF_arr)    
    hydraulic_power_uncertainty = sqrt_sumsq([ hydraulic_power_BUF_mean, 
                                               Precision_Uncertainty(hydraulic_power_arr) ])
    hydraulic_power_mean = np.nanmean(hydraulic_power_arr)
    #Finished hydraulic power calculation
    
    
    #Calculating efficiency
    efficiency_arr = hydraulic_power_arr / shaft_power_arr
    
    efficiency_BUF_arr = []
    for i in range(len(efficiency_arr)):
        efficiency_UPE = sqrt_sumsq([  hydraulic_power_BUF_arr[i] / shaft_power_arr[i], 
                                       -1 * hydraulic_power_arr[i] / shaft_power_arr[i]**2 * shaft_power_BUF_arr[i] ])
        efficiency_BUF_arr.append(efficiency_UPE)
    
    efficiency_BUF_mean = np.nanmean(efficiency_BUF_arr)
    efficiency_uncertainty = sqrt_sumsq([ Precision_Uncertainty(efficiency_arr) , efficiency_BUF_mean ])
    efficiency_mean = np.nanmean(efficiency_arr)
    
    
    dPt_mean = dPt_mean * 100000 #Convert to Pa
    dPt_uncertainty = dPt_uncertainty * 100000 #Convert to Pa
    
    #Convert to Pi-Group
    N_mean = grand_means['rs_mean'] * 2 * np.pi / 60
    D = [50.00 / 1000, 50.04 / 1000, 50.00 / 1000, 50.02 / 1000, 50.02 / 1000]
    D_mean = np.nanmean(D)
    
    U_D = sqrt_sumsq([Bias_Uncertainty(vernier_acc,vernier_res), Precision_Uncertainty(D)])
    
    viscosity = [25.3 / 1000 , 25.4 / 1000 , 25.45 / 1000 , 25.25 / 1000 , 25.15 / 1000]
    viscosity_mean = np.nanmean(viscosity)
    viscometer_acc = 0.5 / 1000
    viscometer_res = 0.1 / 1000
    viscosity_uncertainty = sqrt_sumsq([ Bias_Uncertainty(viscometer_acc,viscometer_res), Precision_Uncertainty(viscosity) ])
    
    #Pi Groups Calculation
    pi_mfr = grand_means['mfr_mean'] / ( D_mean**3 * N_mean * density )
    pi_reynolds = (D_mean**2 * N_mean * density) / viscosity_mean
    pi_dPt = dPt_mean / (D_mean**2 * N_mean**2 * density)
    pi_shaft_power = shaft_power_mean / (D_mean**5 * N_mean**3 * density)
    
    #Pi Groups UPE
    pi_mfr_uncertainty = sqrt_sumsq([
        mfr_uncertainty / ( D_mean**3 * N_mean * density ),
        -3 * grand_means['mfr_mean'] / ( D_mean**4 * N_mean * density ) * U_D,
        -1 * grand_means['mfr_mean'] / ( D_mean**3 * N_mean**2 * density ) * rs_uncertainty,
        -1 * grand_means['mfr_mean'] / ( D_mean**3 * N_mean * density**2 ) * density_uncertainty
    ])
    
    pi_reynolds_uncertainty = sqrt_sumsq([
        viscosity_uncertainty / (D_mean**2 * N_mean * density),
        -2 * viscosity_mean / (D_mean**3 * N_mean * density) * U_D,
        -1 * viscosity_mean / (D_mean**2 * N_mean**2 * density) * rs_uncertainty,
        -1 * viscosity_mean / (D_mean**2 * N_mean * density**2) * density_uncertainty
    ])
    
    pi_reynolds_uncertainty = sqrt_sumsq([
        -1 * density * N_mean * D_mean**2 / viscosity_mean**2 * viscosity_uncertainty,
        N_mean * D_mean**2 / viscosity_mean * density_uncertainty,
        density * D_mean**2 / viscosity_mean * rs_uncertainty,
        2 * density * N_mean * D_mean / viscosity_mean * U_D
    ])
    
    pi_dPt_uncertainty = sqrt_sumsq([
        dPt_uncertainty / (D_mean**2 * N_mean**2 * density ),
        -2 * dPt_mean / (D_mean**3 * N_mean**2 * density) * U_D,
        -2 * dPt_mean / (D_mean**2 * N_mean**3 * density) * rs_uncertainty,
        -1 * dPt_mean / (D_mean**2 * N_mean**2 * density**2) * density_uncertainty
    ])
    
    pi_shaft_power_uncertainty = sqrt_sumsq([
        shaft_power_uncertainty / (D_mean**5 * N_mean**3 * density),
        -5 * shaft_power_mean / (D_mean**6 * N_mean**3 * density) * U_D,
        -3 * shaft_power_mean / (D_mean**5 * N_mean**4 * density) * rs_uncertainty,
        -1 * shaft_power_mean / (D_mean**5 * N_mean**3 * density**2) * density_uncertainty
    ])
    
    dPt_mean = dPt_mean / 100000 #Convert back to bar
    dPt_uncertainty = dPt_uncertainty / 100000 #Convert back to bar
    
    #Prototype Convert
    D_prototype = 50/1000 #50 mm
    
    if grand_means['rs_mean'] > 7700 :
        N_prototype = 15090 * 2 * np.pi / 60
    elif grand_means['rs_mean'] > 7000:
        N_prototype = 14090 * 2 * np.pi / 60
    elif grand_means['rs_mean'] < 7000:
        N_prototype = 13090 * 2 * np.pi / 60
    else:
        print("Speed Error")
        
    density_prototype = 1553.6
    
    mfr_prototype = pi_mfr * D_prototype**3 * N_prototype * density_prototype
    dPt_prototype = pi_dPt * D_prototype**2 * N_prototype**2 * density_prototype
    shaft_power_prototype = pi_shaft_power * D_prototype**5 * N_prototype**3 * density_prototype
    
    mfr_prototype_uncertainty = pi_mfr_uncertainty * D_prototype**3 * N_prototype * density_prototype
    dPt_prototype_uncertainty = pi_dPt_uncertainty * D_prototype**2 * N_prototype**2 * density_prototype
    shaft_power_prototype_uncertainty = pi_shaft_power_uncertainty * D_prototype**5 * N_prototype**3 * density_prototype
    
    dPt_prototype = dPt_prototype / 100000 #Convert back to bar
    dPt_prototype_uncertainty = dPt_prototype_uncertainty / 100000 #Convert back to bar
    

    # --- PRINT ---
    print(f"\nTime-Averaged Values:")
    for s in summaries:
        print(f"{s['file']}: Mass Flow Rate={s['mass_flow_rate']:.5f}, "
                    f"ΔPs={s['mean_dp']:.5f}, RPM={s['mean_rs']:.2f}, "
                    f"Torque={s['mean_torque']:.5f}")
            
    print(f"\nTime-Averaged S.D. / S.E.:")
        
    for s in summaries:
        print(
                f"{s['file']}: "
                f"SE (95% CI) Mass Flow Rate={s['se_mass_flow_rate']:.5f}, "
                f"1.96*SD ΔPs={s['std_dp']:.5f}, "
                f"1.96*SD RPM={s['std_rs']:.5f}, "
                f"1.96*SD Torque={s['std_tq']:.5f}"
        )
        
    print(f"\nInstance Average of Means Value:")
    print(f"ΔPs:                 mean={grand_means['dp_mean']:.5f}, SD={grand_means['dp_sd']:.5f}, Precision Uncertainty={grand_pu['dp_pu']:.5f}, Bias Uncertainty={dp_bu:.5f}")
    print(f"RPM:                 mean={grand_means['rs_mean']:.2f}, SD={grand_means['rs_sd']:.2f}, Precision Uncertainty={grand_pu['rs_pu']:.5f}, Bias Uncertainty={rs_bu:.5f}")
    print(f"Torque:              mean={grand_means['tq_mean']:.5f}, SD={grand_means['tq_sd']:.5f}, Precision Uncertainty={grand_pu['tq_pu']:.5f}, Bias Uncertainty={tq_bu:.5f}\n")
    print(f"Derived Quantities:")
    print(f"Mass Flow Rate:      mean={grand_means['mfr_mean']:.5f}")
    print(f"ΔPt:                 mean={dPt_mean:.5f} Bar")
    print(f"Shaft Power:         mean={shaft_power_mean:.5f} W")
    print(f"Hydraulic Power:     mean={hydraulic_power_mean:.5f} W")
    print(f"Efficiency:          mean={efficiency_mean*100:.2f}%")
        
    print(f"\nInstance Average of Time-Averaged SDs (SD̄):")
    print(f"ΔPs 1.96*SD̄ = {avg_sd['dp_bar']:.5f}")
    print(f"RPM 1.96*SD̄= {avg_sd['rs_bar']:.5f}")
    print(f"Torque 1.96*SD̄ = {avg_sd['tq_bar']:.5f}")

    print(f"\nFinal Uncertainty:")
    print(f"Uncertainty of ΔPs (RAW) = {dp_uncertainty:.5f} Bar")
    print(f"Uncertainty of RPM = {rs_uncertainty:.5f} RPM")
    print(f"Uncertainty of Torque = {tq_uncertainty:.5f} N-m\n")
    print(f"Derived Quantities:")
    print(f"Uncertainty of Mass Flow Rate = {mfr_uncertainty:.5f} Kg/s")
    print(f"Uncertainty of ΔPt = {dPt_uncertainty:.8f} Bar")
    print(f"Uncertainty of Shaft Power = {shaft_power_uncertainty:.5f} W")
    print(f"Uncertainty of Hydraulic Power = {hydraulic_power_uncertainty:.5f} W")
    print(f"Uncertainty of Efficiency = {efficiency_uncertainty*100:.5f} %")


    # --- WRITE summary.txt ---
    folder = os.path.basename(csv_dir.rstrip(os.sep))
    summary_path = os.path.join(csv_dir, f"{folder} summary.txt")
    database_path = os.path.join(csv_dir, f"{folder} data.txt")
    with open(summary_path, 'w', encoding='utf-8') as f:
        
        f.write("\nTime-Averaged Values:\n")
        for s in summaries:
            f.write(f"{s['file']}: Mass Flow Rate={s['mass_flow_rate']:.5f}, "
                    f"ΔPs={s['mean_dp']:.5f}, RPM={s['mean_rs']:.2f}, "
                    f"Torque={s['mean_torque']:.5f}\n")
            
        f.write("\nTime-Averaged S.D. / S.E.:\n")
        
        for s in summaries:
            f.write(
                f"{s['file']}: "
                f"SE (95% CI) Mass Flow Rate={s['se_mass_flow_rate']:.5f}, "
                f"1.96*SD ΔPs={s['std_dp']:.5f}, "
                f"1.96*SD RPM={s['std_rs']:.5f}, "
                f"1.96*SD Torque={s['std_tq']:.5f}\n"
            )
        
        f.write("\nInstance Average of Means Value:\n")
        f.write(f"ΔPs:                 mean={grand_means['dp_mean']:.5f}, SD={grand_means['dp_sd']:.5f}, Precision Uncertainty={grand_pu['dp_pu']:.5f}, Bias Uncertainty={dp_bu:.5f}\n")
        f.write(f"RPM:                 mean={grand_means['rs_mean']:.2f}, SD={grand_means['rs_sd']:.2f}, Precision Uncertainty={grand_pu['rs_pu']:.5f}, Bias Uncertainty={rs_bu:.5f}\n")
        f.write(f"Torque:              mean={grand_means['tq_mean']:.5f}, SD={grand_means['tq_sd']:.5f}, Precision Uncertainty={grand_pu['tq_pu']:.5f}, Bias Uncertainty={tq_bu:.5f}\n\n")
        f.write("Derived Quantities:\n")
        f.write(f"Mass Flow Rate:      mean={grand_means['mfr_mean']:.5f} Kg/s\n")
        f.write(f"ΔPt:                 mean={dPt_mean:.5f} Bar\n")
        f.write(f"Shaft Power:         mean={shaft_power_mean:.5f} W\n")
        f.write(f"Hydraulic Power:     mean={hydraulic_power_mean:.5f} W\n")
        f.write(f"Efficiency:          mean={efficiency_mean*100:.2f}%\n")
        
        f.write("\nUncertainties:\n")
        f.write(f"Uncertainty of ΔPs (RAW) = {dp_uncertainty:.5f} Bar  |  {dp_uncertainty/grand_means['dp_mean']*100:.2f}%\n")
        f.write(f"Uncertainty of RPM = {rs_uncertainty:.5f} RPM  |  {rs_uncertainty/grand_means['rs_mean']*100:.2f}%\n")
        f.write(f"Uncertainty of Torque = {tq_uncertainty:.5f} N-m  |  {dp_uncertainty/grand_means['tq_mean']*100:.2f}%\n\n")
        f.write("Derived Quantities:\n")
        f.write(f"Uncertainty of Mass Flow Rate = {mfr_uncertainty:.5f} Kg/s  |  {mfr_uncertainty/grand_means['mfr_mean']*100:.2f}%\n")
        f.write(f"Uncertainty of ΔPt = {dPt_uncertainty:.5f} Bar  |  {dPt_uncertainty/dPt_mean*100:.2f}%\n")
        f.write(f"Uncertainty of Shaft Power = {shaft_power_uncertainty:.5f} W  |  {shaft_power_uncertainty/shaft_power_mean*100:.2f}%\n")
        f.write(f"Uncertainty of Hydraulic Power = {hydraulic_power_uncertainty:.5f} W  |  {hydraulic_power_uncertainty/hydraulic_power_mean*100:.2f}%\n")
        f.write(f"Uncertainty of Efficiency = {efficiency_uncertainty*100:.5f} %  |  {efficiency_uncertainty/efficiency_mean*100:.2f}%\n")
        
        f.write("\nPi Group Conversion:\n")
        f.write(f"PI Mass Flow Rate:      {pi_mfr:.5f}\n")
        f.write(f"PI ΔPt:                 {pi_dPt:.5f}\n")
        f.write(f"PI Shaft Power:         {pi_shaft_power:.5f}\n")
        f.write(f"Efficiency:             {efficiency_mean*100:.2f}%\n")
        
        f.write("\nPi Group Uncertainty:\n")
        f.write(f"Uncertainty of PI Mass Flow Rate:      {pi_mfr_uncertainty:.5f}  |  {pi_mfr_uncertainty/pi_mfr*100:.2f}%\n")
        f.write(f"Uncertainty of PI ΔPt:                 {pi_dPt_uncertainty:.5f})  |  {pi_dPt_uncertainty/pi_dPt*100:.2f}%\n")
        f.write(f"Uncertainty of PI Shaft Power:         {pi_shaft_power_uncertainty:.5f}  |  {pi_shaft_power_uncertainty/pi_shaft_power*100:.2f}%\n")
        f.write(f"Uncertainty of Efficiency:             {efficiency_uncertainty*100:.2f}%  |  {efficiency_uncertainty/efficiency_mean*100:.2f}%\n")
        
        f.write("\nPrototype Conversion:\n")
        f.write(f"Prototype Mass Flow Rate:      {mfr_prototype:.5f} \n")
        f.write(f"Prototype ΔPt:                 {dPt_prototype:.5f} Bar\n")
        f.write(f"Prototype Shaft Power:         {shaft_power_prototype:.5f} W\n")
        f.write(f"Prototype Efficiency:             {efficiency_mean*100:.2f}%\n")
        
        f.write("\nPrototype Uncertainty:\n")
        f.write(f"Uncertainty of Prototype Mass Flow Rate:      {mfr_prototype_uncertainty:.5f} Kg/s  |  {pi_mfr_uncertainty/pi_mfr*100:.2f}%\n")
        f.write(f"Uncertainty of Prototype ΔPt:                 {dPt_prototype_uncertainty:.5f} Bar  |  {pi_dPt_uncertainty/pi_dPt*100:.2f}%\n")
        f.write(f"Uncertainty of Prototype Shaft Power:         {shaft_power_prototype_uncertainty:.5f} W  |  {pi_shaft_power_uncertainty/pi_shaft_power*100:.2f}%\n")
        f.write(f"Uncertainty of Prototype Efficiency:             {efficiency_uncertainty*100:.2f}%  |  {efficiency_uncertainty/efficiency_mean*100:.2f}%\n")
        
        
    print(f"Summary written to {summary_path}")
    

if __name__ == '__main__':
    # 1) Base path (up to “Glycerine\”)
    BASE_DIR = r'/Users/krai/Library/CloudStorage/OneDrive-ChulalongkornUniversity/Electric Pump Senior Project/Experimental Record/Glycerine'
    
    # 2) List the final two-level folders you want to process:
    subdirs = [
        r'6730 rpm/Glycerine_08%_6730_0.081Kg-s',
        r'6730 rpm/Glycerine_13%_6730_0.160Kg-s',
        r'6730 rpm/Glycerine_19%_6730_0.235Kg-s',
        r'6730 rpm/Glycerine_21%_6730_0.272Kg-s',
        r'6730 rpm/Glycerine_25.6%_6730_0.302Kg-s',
        r'6730 rpm/Glycerine_31.5%_6730_0.344Kg-s',
        r'6730 rpm/Glycerine_36.5%_6730_0.378Kg-s',
        r'6730 rpm/Glycerine_43%_6730_0.416Kg-s',
        r'6730 rpm/Glycerine_52.5%_6730_0.454Kg-s',
        r'6730 rpm/Glycerine_85%_6730_0.526Kg-s',
        r'7250 rpm/Glycerine_08%_7250_0.094Kg-s',
        r'7250 rpm/Glycerine_13%_7250_0.175Kg-s',
        r'7250 rpm/Glycerine_19%_7250_0.253Kg-s',
        r'7250 rpm/Glycerine_21%_7250_0.290Kg-s',
        r'7250 rpm/Glycerine_25.6%_7250_0.328Kg-s',
        r'7250 rpm/Glycerine_31.5%_7250_0.370Kg-s',
        r'7250 rpm/Glycerine_36.5%_7250_0.408Kg-s',
        r'7250 rpm/Glycerine_43%_7250_0.447Kg-s',
        r'7250 rpm/Glycerine_52.5%_7250_0.487Kg-s',
        r'7250 rpm/Glycerine_85%_7250_0.565Kg-s',
        r'7760 rpm/Glycerine_08%_7760_0.094Kg-s',
        r'7760 rpm/Glycerine_13%_7760_0.187Kg-s',
        r'7760 rpm/Glycerine_19%_7760_0.268Kg-s',
        r'7760 rpm/Glycerine_21%_7760_0.313Kg-s',
        r'7760 rpm/Glycerine_27.5%_7760_0.354Kg-s',
        r'7760 rpm/Glycerine_31.5%_7760_0.395Kg-s',
        r'7760 rpm/Glycerine_36.5_7760_0.437Kg-s',
        r'7760 rpm/Glycerine_43%_7760_0.477Kg-s',
        r'7760 rpm/Glycerine_52.5%_7760_0.521Kg-s',
        r'7760 rpm/Glycerine_85%_7760_0.603Kg-s'
    ]

    # 3) Iterate through each and run
    for sub in subdirs:
        csv_directory = os.path.join(BASE_DIR, sub)
        print(f"\n=== Processing folder: {csv_directory} ===")
        main(csv_directory)



=== Processing folder: /Users/krai/Library/CloudStorage/OneDrive-ChulalongkornUniversity/Electric Pump Senior Project/Experimental Record/Glycerine/24042025/Glycerine_8%_6730_0.081Kg-s ===

Time-Averaged Values:
Glycerine_24042025_1354_6730RPM_8%_#1_0.081Kg-s: Mass Flow Rate=0.07974, ΔPs=1.99306, RPM=6729.90, Torque=0.33636
Glycerine_24042025_1412_6730RPM_8%_#2_0.081Kg-s: Mass Flow Rate=0.08020, ΔPs=1.99871, RPM=6730.00, Torque=0.33042
Glycerine_24042025_1506_6730RPM_8%_#5_0.079Kg-s: Mass Flow Rate=0.07872, ΔPs=2.00722, RPM=6729.99, Torque=0.28646
Glycerine_24042025_1448_6730RPM_8%_#4_0.080Kg-s: Mass Flow Rate=0.07959, ΔPs=2.00494, RPM=6729.99, Torque=0.31523
Glycerine_24042025_1430_6730RPM_8%_#3_0.080Kg-s: Mass Flow Rate=0.07979, ΔPs=1.99867, RPM=6730.06, Torque=0.32723

Time-Averaged S.D. / S.E.:
Glycerine_24042025_1354_6730RPM_8%_#1_0.081Kg-s: SE (95% CI) Mass Flow Rate=0.00002, 1.96*SD ΔPs=0.01798, 1.96*SD RPM=16.28229, 1.96*SD Torque=0.00343
Glycerine_24042025_1412_6730RPM_8%_#2_

In [4]:
 #MACBOOK
#6730 rpm
BASE_DIR = r'/Users/krai/Library/CloudStorage/OneDrive-ChulalongkornUniversity/Electric Pump Senior Project/Experimental Record/Glycerine'

subdirs = [
        r'6730 rpm/Glycerine_08%_6730_0.081Kg-s',
        r'6730 rpm/Glycerine_13%_6730_0.160Kg-s',
        r'6730 rpm/Glycerine_19%_6730_0.235Kg-s',
        r'6730 rpm/Glycerine_21%_6730_0.272Kg-s',
        r'6730 rpm/Glycerine_25.6%_6730_0.302Kg-s',
        r'6730 rpm/Glycerine_31.5%_6730_0.344Kg-s',
        r'6730 rpm/Glycerine_36.5%_6730_0.378Kg-s',
        r'6730 rpm/Glycerine_43%_6730_0.416Kg-s',
        r'6730 rpm/Glycerine_52.5%_6730_0.454Kg-s',
        r'6730 rpm/Glycerine_85%_6730_0.526Kg-s',
        r'7250 rpm/Glycerine_08%_7250_0.094Kg-s',
        r'7250 rpm/Glycerine_13%_7250_0.175Kg-s',
        r'7250 rpm/Glycerine_19%_7250_0.253Kg-s',
        r'7250 rpm/Glycerine_21%_7250_0.290Kg-s',
        r'7250 rpm/Glycerine_25.6%_7250_0.328Kg-s',
        r'7250 rpm/Glycerine_31.5%_7250_0.370Kg-s',
        r'7250 rpm/Glycerine_36.5%_7250_0.408Kg-s',
        r'7250 rpm/Glycerine_43%_7250_0.447Kg-s',
        r'7250 rpm/Glycerine_52.5%_7250_0.487Kg-s',
        r'7250 rpm/Glycerine_85%_7250_0.565Kg-s',
        r'7760 rpm/Glycerine_08%_7760_0.094Kg-s',
        r'7760 rpm/Glycerine_13%_7760_0.187Kg-s',
        r'7760 rpm/Glycerine_19%_7760_0.268Kg-s',
        r'7760 rpm/Glycerine_21%_7760_0.313Kg-s',
        r'7760 rpm/Glycerine_27.5%_7760_0.354Kg-s',
        r'7760 rpm/Glycerine_31.5%_7760_0.395Kg-s',
        r'7760 rpm/Glycerine_36.5_7760_0.437Kg-s',
        r'7760 rpm/Glycerine_43%_7760_0.477Kg-s',
        r'7760 rpm/Glycerine_52.5%_7760_0.521Kg-s',
        r'7760 rpm/Glycerine_85%_7760_0.603Kg-s'
    ]

#Windows
BASE_DIR = r'C:\Users\Krai\OneDrive - Chulalongkorn University\Electric Pump Senior Project\Experimental Record\Glycerine'

subdirs = [
        r'6730 rpm\Glycerine_08%_6730_0.081Kg-s',
        r'6730 rpm\Glycerine_13%_6730_0.160Kg-s',
        r'6730 rpm\Glycerine_19%_6730_0.235Kg-s',
        r'6730 rpm\Glycerine_21%_6730_0.272Kg-s',
        r'6730 rpm\Glycerine_25.6%_6730_0.302Kg-s',
        r'6730 rpm\Glycerine_31.5%_6730_0.344Kg-s',
        r'6730 rpm\Glycerine_36.5%_6730_0.378Kg-s',
        r'6730 rpm\Glycerine_43%_6730_0.416Kg-s',
        r'6730 rpm\Glycerine_52.5%_6730_0.454Kg-s',
        r'6730 rpm\Glycerine_85%_6730_0.526Kg-s',
        r'7250 rpm\Glycerine_08%_7250_0.094Kg-s',
        r'7250 rpm\Glycerine_13%_7250_0.175Kg-s',
        r'7250 rpm\Glycerine_19%_7250_0.253Kg-s',
        r'7250 rpm\Glycerine_21%_7250_0.290Kg-s',
        r'7250 rpm\Glycerine_25.6%_7250_0.328Kg-s',
        r'7250 rpm\Glycerine_31.5%_7250_0.370Kg-s',
        r'7250 rpm\Glycerine_36.5%_7250_0.408Kg-s',
        r'7250 rpm\Glycerine_43%_7250_0.447Kg-s',
        r'7250 rpm\Glycerine_52.5%_7250_0.487Kg-s',
        r'7250 rpm\Glycerine_85%_7250_0.565Kg-s',
        r'7760 rpm\Glycerine_08%_7760_0.094Kg-s',
        r'7760 rpm\Glycerine_13%_7760_0.187Kg-s',
        r'7760 rpm\Glycerine_19%_7760_0.268Kg-s',
        r'7760 rpm\Glycerine_21%_7760_0.313Kg-s',
        r'7760 rpm\Glycerine_27.5%_7760_0.354Kg-s',
        r'7760 rpm\Glycerine_31.5%_7760_0.395Kg-s',
        r'7760 rpm\Glycerine_36.5_7760_0.437Kg-s',
        r'7760 rpm\Glycerine_43%_7760_0.477Kg-s',
        r'7760 rpm\Glycerine_52.5%_7760_0.521Kg-s',
        r'7760 rpm\Glycerine_85%_7760_0.603Kg-s'
    ]

IndentationError: unindent does not match any outer indentation level (<string>, line 44)

In [ ]:
import os
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import linregress
import matplotlib.font_manager as font_manager

def process_file(file_path):
    font_legend = {'family': 'TH Sarabun New',
                    'weight': 'bold',
                    'style': 'normal', 
                    'size': 18
                   }
    font_axis = {'family': 'TH Sarabun New',
                    'weight': 'bold',
                    'style': 'normal', 
                    'size': 20
                   }
    base_name   = os.path.splitext(os.path.basename(file_path))[0]
    parent_dir  = os.path.dirname(file_path)
    output_dir  = os.path.join(parent_dir, f"{base_name} plots")
    output_dir  = os.path.join(output_dir, f"combined plots")
    os.makedirs(output_dir, exist_ok=True)

    # Read CSV, skip first two summary rows, use third row as header
    df = pd.read_csv(file_path, delimiter=',', skiprows=2, header=0)

    # --- COMBINED FULL PLOTS ---

    # 1) Torque and Pressure vs Weight
    x = df['Weight [Kg]']
    #y = df['Weight [Kg]']

    fig, ax1 = plt.subplots()
    ax2 = ax1.twinx()
    #plt.plot(x, y, label='Data')
    #plt.xlabel('Run Time [s]')
    #plt.ylabel('Weight [Kg]')
    #plt.title('Mass Flow Rate vs Run Time')
    #plt.legend()
    #plt.ylim(bottom=0)
    #plt.tight_layout()

    lns1 = ax1.plot(x, df['Differential Pressure [Bar]'],label='Differential Pressure')
    ax1.set_xlabel('Weight [Kg]', fontdict=font_axis)
    ax1.set_ylabel('Differential Pressure [Bar]', fontdict=font_axis)
    ax1.set_ylim(bottom=0)

    lns2 = ax2.plot(x, df["Torque [N-m]"],color="orange",label='Torque')
    ax2.set_ylabel('Torque [N-m]', fontdict=font_axis)
    ax2.set_ylim([0,0.6])
    
    lns = lns1 + lns2
    
    labs = [l.get_label() for l in lns]       
    ax1.legend(lns, labs, loc="lower right",prop=font_legend)
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, f"{base_name}_Pressure_&_Torque_vs_Weight_full_combined_plot.png"))
    plt.close()
    
    # 2) Torque and Pressure vs Time
    x = df['Run Time [s]']
    #y = df['Weight [Kg]']

    fig, ax1 = plt.subplots()
    ax2 = ax1.twinx()
    #plt.plot(x, y, label='Data')
    #plt.xlabel('Run Time [s]')
    #plt.ylabel('Weight [Kg]')
    #plt.title('Mass Flow Rate vs Run Time')
    #plt.legend()
    #plt.ylim(bottom=0)
    #plt.tight_layout()

    lns1 = ax1.plot(x, df['Differential Pressure [Bar]'],label='Differential Pressure')
    ax1.set_xlabel('Run Time [s]', fontdict=font_axis)
    ax1.set_ylabel('Differential Pressure [Bar]', fontdict=font_axis)
    ax1.set_ylim(bottom=0)
    
    lns2 = ax2.plot(x, df["Torque [N-m]"],color="orange",label='Torque')
    ax2.set_ylabel('Torque [N-m]',fontsize = 16, fontdict=font_axis)
    ax2.set_ylim([0,0.6])
    
    lns = lns1 + lns2

    labs = [l.get_label() for l in lns]       
    ax1.legend(lns, labs, loc="lower right",prop=font_legend)
    
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, f"{base_name}_Pressure_&_Torque_vs_Run_Time_full_combined_plot.png"))
    plt.close()

def main(csv_dir):
    files = glob.glob(os.path.join(csv_dir, '*.csv'))
    if not files:
        print(f"No CSV files found in {csv_dir}")
        return

    summaries = [process_file(f) for f in files]

    
    
if __name__ == '__main__':
    # 1) Base path (up to “Glycerine\”)
    BASE_DIR = r'C:\Users\Krai\OneDrive - Chulalongkorn University\Electric Pump Senior Project\Experimental Record\Glycerine'
    
    # 2) List the final two-level folders you want to process:
    subdirs = [
        r'24042025\Glycerine_8%_6730_0.081Kg-s',
        r'24042025\Glycerine_8%_7760_0.094Kg-s',
        r'23042025\Glycerine_8%_7250_0.094Kg-s',
        r'23042025\Glycerine_85%_6730_0.526Kg-s',
        r'23042025\Glycerine_85%_7760_0.603Kg-s',
        r'22042025\Glycerine_52.5%_6730_0.454Kg-s',
        r'22042025\Glycerine_52.5%_7760_0.521Kg-s',
        r'22042025\Glycerine_85%_7250_0.565Kg-s',
        r'21042025\Glycerine_43%_6730_0.416Kg-s',
        r'21042025\Glycerine_43%_7250_0.447Kg-s',
        r'21042025\Glycerine_43%_7760_0.477Kg-s',
        r'21042025\Glycerine_52.5%_7250_0.487Kg-s',
        r'20042025\Glycerine_31.5%_6730_0.344Kg-s',
        r'20042025\Glycerine_31.5%_7250_0.370Kg-s',
        r'20042025\Glycerine_36.5%_6730_0.378Kg-s',
        r'20042025\Glycerine_36.5%_7250_0.408Kg-s',
        r'19042025\Glycerine_13%_6730_0.160Kg-s',
        r'19042025\Glycerine_13%_7760_0.188Kg-s',
        r'18042025\Glycerine_13%_7250_0.175Kg-s',
        r'18042025\Glycerine_19%_7760_0.268Kg-s',
        r'17042025\Glycerine_19%_6730_0.235Kg-s',
        r'17042025\Glycerine_19%_7250_0.253Kg-s',
        r'17042025\Glycerine_25.6%_6730_0.302Kg-s',
        r'17042025\Glycerine_25.6%_7250_0.328Kg-s',
        r'16042025\Glycerine_21%_6730_0.272Kg-s',
        r'26042025\Glycerine_27.5%_7760_0.354Kg-s',
        r'26042025\Glycerine_21%_7760_0.313Kg-s',
        r'26042025\Glycerine_21%_7250_0.281Kg-s',
        r'28042025\Glycerine_13%_6730_0.160Kg-s',
        r'28042025\Glycerine_21%_7250_0.290Kg-s',
        r'28042025\Glycerine_31.5%_7760_0.395Kg-s',
        r'28042025\Glycerine_36.5_7760_0.437Kg-s',
        r'28042025\Glycerine_52.5%_7760_0.521Kg-s',
        r'29042025\Glycerine_13%_7760_0.187Kg-s',
    ]

    # 3) Iterate through each and run
    for sub in subdirs:
        csv_directory = os.path.join(BASE_DIR, sub)
        print(f"\n=== Processing folder: {csv_directory} ===")
        main(csv_directory)


        

In [53]:
import os
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import linregress
from scipy.stats import t
import sympy as sp

def sqrt_sumsq(arr):
    place_holder = []
    for i in arr:
        place_holder.append(i**2)
    return np.sqrt(np.sum(place_holder))

def Precision_Uncertainty(arr):
    
    #Precision Uncertainty Calculation Initialization
    Ns = len(arr)
    dof = Ns - 1  # degrees of freedom for mean
    t_crit = t.ppf(0.975, dof) #0.975 for two-tails student-t
    
    Prec_Unc = t_crit * np.nanstd(arr, ddof=1) / np.sqrt(Ns)
    return Prec_Unc

def Bias_Uncertainty(Acc, Res):
    
    Bias_Unc = sqrt_sumsq([Acc, Res/2])
    
    return Bias_Unc

def UPE_dPt(
    dPs_val, U_dPs_val,
    mass_val, U_mass_val,
    vol_val,  U_vol_val,
    dh_val,   U_dh_val,
    m_dot_val, U_mdot_val,
    A1_val, A2_val, U_A1_val, U_A2_val,
    g_val=9.807
):
    # 1) Symbolic setup with mass and volume
    dPs, mass, vol, g, dh, m_dot, A1, A2 = sp.symbols('dPs mass vol g dh m_dot A1 A2', real=True)
    U_dPs, U_mass, U_vol, U_dh, U_mdot, U_A1, U_A2 = sp.symbols('U_dPs U_mass U_vol U_dh U_mdot U_A1 U_A2', real=True, positive=True)
    
    # 2) Define rho = mass/vol and the formula for dPt
    rho = mass/vol
    dPt = (
        dPs
        + rho * g * dh
        + sp.Rational(1,2) * rho * ((m_dot/(rho*A2))**2 - (m_dot/(rho*A1))**2)
    )
    
    # 3) Compute symbolic partial derivatives w.r.t variables
    vars_sym = (dPs, mass, vol, dh, m_dot, A1, A2)
    partials = [sp.diff(dPt, var) for var in vars_sym]
    
    # 4) Build combined uncertainty U_dPt
    U_vars = (U_dPs, U_mass, U_vol, U_dh, U_mdot, U_A1, U_A2)
    U_dPt = sp.sqrt(sum((partials[i] * U_vars[i])**2 for i in range(len(vars_sym))))
    U_dPt_simplified = sp.simplify(U_dPt)
    
    # 5) Lambdify for numeric evaluation
    f_dPt      = sp.lambdify((dPs, mass, vol, g, dh, m_dot, A1, A2), dPt, 'numpy')
    f_partials = [sp.lambdify((dPs, mass, vol, g, dh, m_dot, A1, A2), pd, 'numpy') for pd in partials]

    # nominal result
    dPt_val = f_dPt(dPs_val, mass_val, vol_val, g_val, dh_val, m_dot_val, A1_val, A2_val)
    
    # compute gradients
    grads = np.array([f(dPs_val, mass_val, vol_val, g_val, dh_val, m_dot_val, A1_val, A2_val)
                      for f in f_partials], dtype=float)
    
    # uncertainties array
    U_vals = np.array([U_dPs_val, U_mass_val, U_vol_val, U_dh_val, U_mdot_val, U_A1_val, U_A2_val], dtype=float)
    
    # RSS propagation
    U_dPt_val = np.sqrt(np.sum((grads * U_vals)**2))
    return dPt_val, U_dPt_val, grads


def process_file(file_path):
    font_legend = {'family': 'TH Sarabun New',
                   'weight': 'bold',
                   'style': 'normal',
                   'size': 18}
    font_axis = {'family': 'TH Sarabun New',
                 'weight': 'bold',
                 'style': 'normal',
                 'size': 20}

    base_name  = os.path.splitext(os.path.basename(file_path))[0]
    parent_dir = os.path.dirname(file_path)
    output_dir = os.path.join(parent_dir, f"{base_name} plots")
    os.makedirs(output_dir, exist_ok=True)

    # Read CSV, skip first two summary rows, use third row as header
    df = pd.read_csv(file_path, delimiter=',', skiprows=2, header=0)
    x  = df['Run Time [s]']

    # 1) Mass Flow Rate vs Run Time (full) — raw data only, no curve fit
    plt.figure()
    plt.plot(x, df['Weight [Kg]'], label='Data')
    plt.xlabel('Run Time [s]', fontdict=font_axis)
    plt.ylabel('Weight [Kg]', fontdict=font_axis)
    plt.title('Mass Flow Rate vs Run Time', fontdict=font_axis)
    plt.legend(prop=font_legend)
    plt.ylim(bottom=0)
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, f"{base_name}_Mass_Flow_Rate_full_plot.png"))
    plt.close()

    # ... (Differential Pressure, Rotational Speed, Torque full-plot blocks unchanged) ...

    # --- FILTERED PLOTS (20 ≤ Weight ≤ 30 Kg) ---
    filtered = df[(df['Weight [Kg]'] >= 20) & (df['Weight [Kg]'] <= 30)]
    if not filtered.empty:
        
        frt = filtered['Run Time [s]'].iloc[-1] - filtered['Run Time [s]'].iloc[0]

        # Mass Flow Rate (filtered fit) + capture its standard error
        xf, yf = filtered['Run Time [s]'], filtered['Weight [Kg]']
        slope_f, intercept_f, r_f, _, stderr_f = linregress(xf, yf)
        # convert 1σ stderr into 95% CI half-width
        dof    = len(xf) - 2
        t_crit = t.ppf(0.975, dof)
        stderr_f *= t_crit
        r2_f = r_f**2
        max_mass = max(yf)
        min_mass = min(yf)

        # Differential Pressure stats + plot
        dp     = filtered['Differential Pressure [Bar]']
        avg_dp = dp.mean()
        std_dp = t_crit * dp.std(ddof=1)

        # Rotational Speed stats + plot
        rs     = filtered['Rotational Speed [RPM]']
        avg_rs = rs.mean()
        std_rs = t_crit * rs.std(ddof=1)

        # Torque stats + plot
        tq     = filtered['Torque [N-m]']
        avg_tq = tq.mean()
        std_tq = t_crit * tq.std(ddof=1)
        
    else:
        slope_f = stderr_f = np.nan
        avg_dp = avg_rs = avg_tq = std_dp = std_rs = std_tq = np.nan
        print(f"{base_name}: no data in weight range 20–30 Kg")

    # ---- RETURN original metrics PLUS S.E. of slope ----
    return {
        'file':                base_name,
        'mass_flow_rate':      slope_f,
        'se_mass_flow_rate':   stderr_f,     # added standard error
        'mean_dp':             avg_dp,
        'mean_rs':             avg_rs,
        'mean_torque':         avg_tq,
        'std_dp':              std_dp,
        'std_rs':              std_rs,
        'std_tq':              std_tq,
        'run_time':            frt,
        'max_mass':            max_mass,
        'min_mass':            min_mass
    }


def main(csv_dir):
    files = glob.glob(os.path.join(csv_dir, '*.csv'))
    if not files:
        print(f"No CSV files found in {csv_dir}")
        return

    summaries = [process_file(f) for f in files]

    # ARRAYS of the PER-FILE time-averaged MEANS (unchanged)
    mfr_arr = np.array([s['mass_flow_rate'] for s in summaries])
    dp_arr  = np.array([s['mean_dp']        for s in summaries])
    rs_arr  = np.array([s['mean_rs']        for s in summaries])
    tq_arr  = np.array([s['mean_torque']    for s in summaries])
    run_time_arr = np.array([s['run_time']    for s in summaries])
    max_mass_arr = np.array([s['max_mass']    for s in summaries])
    min_mass_arr = np.array([s['min_mass']    for s in summaries])
    
    # ARRAYS of the PER-FILE SDs (unchanged)
    sd_dp_arr = np.array([s['std_dp'] for s in summaries])
    sd_rs_arr = np.array([s['std_rs'] for s in summaries])
    sd_tq_arr = np.array([s['std_tq'] for s in summaries])

    # ARRAY of the PER-FILE S.E.
    se_mfr_arr = np.array([s['se_mass_flow_rate'] for s in summaries])

    # GRAND MEANS & SDs of the MEANS (unchanged)
    grand_means = {
        'mfr_mean':  np.nanmean(mfr_arr), 'mfr_sd':  np.nanstd(mfr_arr, ddof=1),
        'dp_mean':   np.nanmean(dp_arr),  'dp_sd':   np.nanstd(dp_arr,  ddof=1),
        'rs_mean':   np.nanmean(rs_arr),  'rs_sd':   np.nanstd(rs_arr,  ddof=1),
        'tq_mean':   np.nanmean(tq_arr),  'tq_sd':   np.nanstd(tq_arr,  ddof=1),
        'run_time_mean':   np.nanmean(run_time_arr),  'run_time_sd':   np.nanstd(run_time_arr,  ddof=1),
        'max_mass_mean':   np.nanmean(max_mass_arr),  'max_mass_sd':   np.nanstd(max_mass_arr,  ddof=1),
        'min_mass_mean':   np.nanmean(min_mass_arr),  'min_mass_sd':   np.nanstd(min_mass_arr,  ddof=1),
    } #Single Number

    # ARITHMETIC AVERAGE OF THE SDs (unchanged)
    avg_sd = {
        'dp_bar': np.nanmean(sd_dp_arr),
        'rs_bar': np.nanmean(sd_rs_arr),
        'tq_bar': np.nanmean(sd_tq_arr),
    }

    # AVERAGE STANDARD ERROR OF MASS FLOW RATE (added)
    #avg_se = np.nanmean(se_mfr_arr)
    
    grand_pu = {
        'mfr_pu':  Precision_Uncertainty(mfr_arr),
        'dp_pu':   Precision_Uncertainty(dp_arr),
        'rs_pu':   Precision_Uncertainty(rs_arr),
        'tq_pu':   Precision_Uncertainty(tq_arr)
    } #Single Number
    
    #Bias Uncertainty Code Area
    Sensor_Accuracy_Dict = {
        'mass_acc': 0.08426150, 'mass_res': 10 / 2**16 * 5, #kg
        'time_acc': 50e-6 * 0.05, 'time_res': 50e-9, #sec
        'dp_acc': 0.0075, 'dp_res': 10 / 2**16 * 0.3, #bar, res = 10V / 16-bit * 0.3 Bar/V, acc = 0.25%FS
        'tq_acc': 0.001, 'tq_res': 10 / 2**16 * 0.1, #N-m, res = 10V / 16-bit * 0.1 N-m/V
        'rs_acc': 0, 'rs_res': 1 #RPM   
    } #Single Number
    mass_bu = Bias_Uncertainty(Sensor_Accuracy_Dict['mass_acc'], Sensor_Accuracy_Dict['mass_res'])
    time_bu = Bias_Uncertainty(Sensor_Accuracy_Dict['time_acc'], Sensor_Accuracy_Dict['time_res'])
    dp_bu = Bias_Uncertainty(Sensor_Accuracy_Dict['dp_acc'], Sensor_Accuracy_Dict['dp_res'])
    rs_bu = Bias_Uncertainty(Sensor_Accuracy_Dict['rs_acc'], Sensor_Accuracy_Dict['rs_res'])
    tq_bu = Bias_Uncertainty(Sensor_Accuracy_Dict['tq_acc'], Sensor_Accuracy_Dict['tq_res'])
    
    
    
    #Derived Quantities Calculation : Total Pressure Rise | Shaft Power | Hydraulic Power | Efficiency
    
    #Individual values
    
    #Calculating mfr uncertainty
    mfr_BUF_arr = []
    mfr_bu = []
    for i in range(len(mfr_arr)):
        mfr_Bias_UPE = sqrt_sumsq([ mass_bu/run_time_arr[i] , mass_bu/run_time_arr[i] , (min_mass_arr[i] - max_mass_arr[i]) / run_time_arr[i]**2 * time_bu ])
        
        mfr_BUF = sqrt_sumsq([ mfr_Bias_UPE, se_mfr_arr[i]])
        mfr_bu.append(mfr_Bias_UPE)
        mfr_BUF_arr.append(mfr_BUF)
    
    mfr_BUF_mean = np.nanmean(mfr_BUF_arr)
    
    
    #Final Sum Uncertainty
    mfr_uncertainty = sqrt_sumsq([grand_pu['mfr_pu'], mfr_BUF_mean])
    dp_uncertainty = sqrt_sumsq([dp_bu, grand_pu['dp_pu'], avg_sd['dp_bar']])
    rs_uncertainty = sqrt_sumsq([rs_bu, grand_pu['rs_pu'], avg_sd['rs_bar']])
    tq_uncertainty = sqrt_sumsq([tq_bu, grand_pu['tq_pu'], avg_sd['tq_bar']])
    
    
    dPt_arr = []
    dPt_BUF_arr = []
    #Denote Bias Uncertainty + Fluctuation = BUF
    
    vernier_acc = 0.03/1000 #0.03 mm
    vernier_res = 0.02/1000 #0.02 mm
    
    #Calculating dPt value and uncertainty
    for i in range(len(dp_arr)):
        dPs, dPs_BUF = dp_arr[i]*100000, (sqrt_sumsq([sd_dp_arr[i], dp_bu]))*100000
        mass = [1.1762, 1.1771, 1.1763, 1.1781, 1.1759]
        mass_mean = np.nanmean(mass) 
        U_mass = sqrt_sumsq([Bias_Uncertainty(0.0001, 0.0001) , Precision_Uncertainty(mass)]) # kg, uncertainty for Density Calculation
        
        vol,  U_vol  = 0.001, Bias_Uncertainty(0.4/1000000 , 0)  # m^3, uncertainty for Density Calculation ±0.3 mL for 1000 mL Volumetric Flask
        
        dh = [0.09304, 0.09288, 0.09296, 0.09300, 0.09306] #m
        dh_mean = np.nanmean(dh)
        U_dh = sqrt_sumsq([Bias_Uncertainty(0.03/1000, 0.02/1000) , Precision_Uncertainty(dh) ])
        
        m_dot, U_mdot = mfr_arr[i], mfr_BUF_arr[i] # kg/s
        
        D1 = [18.08 / 1000, 18.12 / 1000, 18.08 / 1000, 18.14 / 1000, 18.12 / 1000]
        D1_mean = np.nanmean(D1)
        D1_uncertainty = sqrt_sumsq([Bias_Uncertainty(vernier_acc, vernier_res), Precision_Uncertainty(D1)])
        
        D2 = [13.36 / 1000, 13.42 / 1000, 13.38 / 1000, 13.30 / 1000, 13.36 / 1000]
        D2_mean = np.nanmean(D2)
        D2_uncertainty = sqrt_sumsq([Bias_Uncertainty(vernier_acc, vernier_res), Precision_Uncertainty(D2)])
        
        A1 = [(np.pi / 4 * i**2) for i in D1]
        A2 = [(np.pi / 4 * i**2) for i in D2]
        A1_mean = np.nanmean(A1)
        A2_mean = np.nanmean(A2)
        A1_uncertainty = sqrt_sumsq([(np.pi / 2 * D1_mean * D1_uncertainty),Precision_Uncertainty(A1)])
        A2_uncertainty = sqrt_sumsq([(np.pi / 2 * D2_mean * D2_uncertainty),Precision_Uncertainty(A2)])
        
        dPt, dPt_uncertainty , _ = UPE_dPt(
        dPs, dPs_BUF, mass_mean, U_mass, vol, U_vol,
        dh_mean, U_dh, m_dot, U_mdot,
        A1_mean, A2_mean, A1_uncertainty, A2_uncertainty)
            
        dPt_arr.append(dPt)
        dPt_BUF_arr.append(dPt_uncertainty)
          
    dPt_mean = np.nanmean(dPt_arr) #Representative Value
    dPt_BUF_mean = np.nanmean(dPt_BUF_arr) #Used as Representative Bias Uncertainty of dPt
    
    dPt_mean = dPt_mean/100000 #Convert Pa back to Bar
    dPt_uncertainty = sqrt_sumsq([dPt_BUF_mean, Precision_Uncertainty(dPt_arr)])/100000 #Combined BUF and PU
    #Finished dPt Calculation
    
    #Calculating shaft power
    shaft_power_arr = tq_arr * rs_arr * 2*np.pi/60
    
    shaft_power_BUF_arr = []
    for i in range(len(shaft_power_arr)):
        tq_BUF = sqrt_sumsq([ tq_bu ,  sd_tq_arr[i] ]) #Torque BUF of an individual sample
        rs_BUF = sqrt_sumsq([ rs_bu ,  sd_rs_arr[i] ]) #RPM BUF of an individual sample
        shaft_power_UPE = sqrt_sumsq([ rs_arr[i] * 2 * np.pi / 60 * tq_BUF, tq_arr[i] * 2 * np.pi / 60 * rs_BUF])
        shaft_power_BUF_arr.append(shaft_power_UPE)
    
    shaft_power_BUF_mean = np.nanmean(shaft_power_BUF_arr)
    shaft_power_uncertainty = sqrt_sumsq([ shaft_power_BUF_mean, Precision_Uncertainty(shaft_power_arr) ])
    shaft_power_mean = np.nanmean(shaft_power_arr)
    
    mass = [1.1762, 1.1771, 1.1763, 1.1781, 1.1759]
    mass_mean = np.nanmean(mass) 
    U_mass = sqrt_sumsq([Bias_Uncertainty(0.0001, 0.0001) , Precision_Uncertainty(mass)]) # kg, uncertainty for Density Calculation
    vol,  U_vol  = 0.001, Bias_Uncertainty(0.4/1000000 , 0)  # m^3, uncertainty for Density Calculation ±0.3 mL for 1000 mL Volumetric Flask
    # Density Uncertainty Propagation Equation
    density = mass_mean / vol
    density_uncertainty = sqrt_sumsq([U_mass / vol, -1 * mass_mean/vol**2 * U_vol])
    #Finished shaft power calculation
    
    #Calculating hydraulic power
    hydraulic_power_arr = dPt_arr * (mfr_arr / density)
    
    hydraulic_power_BUF_arr = []
    for i in range(len(hydraulic_power_arr)):
        #dPt_arr + dPt_BUF_arr
        #density + density_uncertainty
        #mfr_arr + se_mfr_arr
        hydraulic_power_UPE = sqrt_sumsq([ mfr_arr[i] / density * dPt_BUF_arr[i],   
                                           dPt_arr[i] / density * mfr_BUF_arr[i], 
                                           -1 * dPt_arr[i] *  mfr_arr[i] / density**2 * density_uncertainty])
        hydraulic_power_BUF_arr.append(hydraulic_power_UPE)
    
    hydraulic_power_BUF_mean = np.nanmean(hydraulic_power_BUF_arr)    
    hydraulic_power_uncertainty = sqrt_sumsq([ hydraulic_power_BUF_mean, 
                                               Precision_Uncertainty(hydraulic_power_arr) ])
    hydraulic_power_mean = np.nanmean(hydraulic_power_arr)
    #Finished hydraulic power calculation
    
    
    #Calculating efficiency
    efficiency_arr = hydraulic_power_arr / shaft_power_arr
    
    efficiency_BUF_arr = []
    for i in range(len(efficiency_arr)):
        efficiency_UPE = sqrt_sumsq([  hydraulic_power_BUF_arr[i] / shaft_power_arr[i], 
                                       -1 * hydraulic_power_arr[i] / shaft_power_arr[i]**2 * shaft_power_BUF_arr[i] ])
        efficiency_BUF_arr.append(efficiency_UPE)
    
    efficiency_BUF_mean = np.nanmean(efficiency_BUF_arr)
    efficiency_uncertainty = sqrt_sumsq([ Precision_Uncertainty(efficiency_arr) , efficiency_BUF_mean ])
    efficiency_mean = np.nanmean(efficiency_arr)
    
    
    dPt_mean = dPt_mean * 100000 #Convert to Pa
    dPt_uncertainty = dPt_uncertainty * 100000 #Convert to Pa
    
    #Convert to Pi-Group
    N_mean = grand_means['rs_mean'] * 2 * np.pi / 60
    D = [50.00 / 1000, 50.04 / 1000, 50.00 / 1000, 50.02 / 1000, 50.02 / 1000]
    D_mean = np.nanmean(D)
    
    U_D = sqrt_sumsq([Bias_Uncertainty(vernier_acc,vernier_res), Precision_Uncertainty(D)])
    
    viscosity = [25.3 / 1000 , 25.4 / 1000 , 25.45 / 1000 , 25.25 / 1000 , 25.15 / 1000]
    viscosity_mean = np.nanmean(viscosity)
    viscometer_acc = 0.5 / 1000
    viscometer_res = 0.1 / 1000
    viscosity_uncertainty = sqrt_sumsq([ Bias_Uncertainty(viscometer_acc,viscometer_res), Precision_Uncertainty(viscosity) ])
    
    #Pi Groups Calculation
    pi_mfr = grand_means['mfr_mean'] / ( D_mean**3 * N_mean * density )
    pi_reynolds = (D_mean**2 * N_mean * density) / viscosity_mean
    pi_dPt = dPt_mean / (D_mean**2 * N_mean**2 * density)
    pi_shaft_power = shaft_power_mean / (D_mean**5 * N_mean**3 * density)
    
    #Pi Groups UPE
    pi_mfr_uncertainty = sqrt_sumsq([
        mfr_uncertainty / ( D_mean**3 * N_mean * density ),
        -3 * grand_means['mfr_mean'] / ( D_mean**4 * N_mean * density ) * U_D,
        -1 * grand_means['mfr_mean'] / ( D_mean**3 * N_mean**2 * density ) * rs_uncertainty,
        -1 * grand_means['mfr_mean'] / ( D_mean**3 * N_mean * density**2 ) * density_uncertainty
    ])
    
    pi_reynolds_uncertainty = sqrt_sumsq([
        viscosity_uncertainty / (D_mean**2 * N_mean * density),
        -2 * viscosity_mean / (D_mean**3 * N_mean * density) * U_D,
        -1 * viscosity_mean / (D_mean**2 * N_mean**2 * density) * rs_uncertainty,
        -1 * viscosity_mean / (D_mean**2 * N_mean * density**2) * density_uncertainty
    ])
    
    pi_reynolds_uncertainty = sqrt_sumsq([
        -1 * density * N_mean * D_mean**2 / viscosity_mean**2 * viscosity_uncertainty,
        N_mean * D_mean**2 / viscosity_mean * density_uncertainty,
        density * D_mean**2 / viscosity_mean * rs_uncertainty,
        2 * density * N_mean * D_mean / viscosity_mean * U_D
    ])
    
    pi_dPt_uncertainty = sqrt_sumsq([
        dPt_uncertainty / (D_mean**2 * N_mean**2 * density ),
        -2 * dPt_mean / (D_mean**3 * N_mean**2 * density) * U_D,
        -2 * dPt_mean / (D_mean**2 * N_mean**3 * density) * rs_uncertainty,
        -1 * dPt_mean / (D_mean**2 * N_mean**2 * density**2) * density_uncertainty
    ])
    
    pi_shaft_power_uncertainty = sqrt_sumsq([
        shaft_power_uncertainty / (D_mean**5 * N_mean**3 * density),
        -5 * shaft_power_mean / (D_mean**6 * N_mean**3 * density) * U_D,
        -3 * shaft_power_mean / (D_mean**5 * N_mean**4 * density) * rs_uncertainty,
        -1 * shaft_power_mean / (D_mean**5 * N_mean**3 * density**2) * density_uncertainty
    ])
    
    dPt_mean = dPt_mean / 100000 #Convert back to bar
    dPt_uncertainty = dPt_uncertainty / 100000 #Convert back to bar
    
    #Prototype Convert
    D_prototype = 50/1000 #50 mm
    
    if grand_means['rs_mean'] > 7700 :
        N_prototype = 15090 * 2 * np.pi / 60
    elif grand_means['rs_mean'] > 7000:
        N_prototype = 14090 * 2 * np.pi / 60
    elif grand_means['rs_mean'] < 7000:
        N_prototype = 13090 * 2 * np.pi / 60
    else:
        print("Speed Error")
        
    density_prototype = 1553.6
    
    mfr_prototype = pi_mfr * D_prototype**3 * N_prototype * density_prototype
    dPt_prototype = pi_dPt * D_prototype**2 * N_prototype**2 * density_prototype
    shaft_power_prototype = pi_shaft_power * D_prototype**5 * N_prototype**3 * density_prototype
    
    mfr_prototype_uncertainty = pi_mfr_uncertainty * D_prototype**3 * N_prototype * density_prototype
    dPt_prototype_uncertainty = pi_dPt_uncertainty * D_prototype**2 * N_prototype**2 * density_prototype
    shaft_power_prototype_uncertainty = pi_shaft_power_uncertainty * D_prototype**5 * N_prototype**3 * density_prototype
    
    dPt_prototype = dPt_prototype / 100000 #Convert back to bar
    dPt_prototype_uncertainty = dPt_prototype_uncertainty / 100000 #Convert back to bar
        
    data_write = f"{grand_means['mfr_mean']:.5f},{dPt_mean:.5f},{grand_means['tq_mean']:.5f},{shaft_power_mean:.5f},{efficiency_mean:.5f},{mfr_uncertainty:.5f},{dPt_uncertainty:.5f},{tq_uncertainty:.5f},{shaft_power_uncertainty:.5f},{efficiency_uncertainty:.5f},{pi_mfr:.5f},{pi_dPt:.5f},{pi_shaft_power:.5f},{pi_reynolds:.5f},{pi_mfr_uncertainty:.5f},{pi_dPt_uncertainty:.5f},{pi_shaft_power_uncertainty:.5f},{pi_reynolds_uncertainty:.10f},{mfr_prototype:.5f},{dPt_prototype:.5f},{shaft_power_prototype:.5f},{mfr_prototype_uncertainty:.5f},{dPt_prototype_uncertainty:.5f},{shaft_power_prototype_uncertainty:.5f}\n"
         
    return data_write, N_prototype  
    

if __name__ == '__main__':
    # 1) Base path (up to “Glycerine\”)
    BASE_DIR = r'/Users/krai/Library/CloudStorage/OneDrive-ChulalongkornUniversity/Electric Pump Senior Project/Experimental Record/Glycerine/7760 rpm'
    
    # 2) List the final two-level folders you want to process:
    subdirs = [
        r'Glycerine_08%_7760_0.094Kg-s',
        r'Glycerine_13%_7760_0.187Kg-s',
        r'Glycerine_19%_7760_0.268Kg-s',
        r'Glycerine_21%_7760_0.313Kg-s',
        r'Glycerine_27.5%_7760_0.354Kg-s',
        r'Glycerine_31.5%_7760_0.395Kg-s',
        r'Glycerine_36.5_7760_0.437Kg-s',
        r'Glycerine_43%_7760_0.477Kg-s',
        r'Glycerine_52.5%_7760_0.521Kg-s',
        r'Glycerine_85%_7760_0.603Kg-s',
    ]
    # 3) Iterate through each and run
    data_write_array = []
    for sub in subdirs:
        csv_directory = os.path.join(BASE_DIR, sub)
        print(f"\n=== Processing folder: {csv_directory} ===")
        data, N_prototype = main(csv_directory)
        data_write_array.append(data)
    
    # --- WRITE summary.txt ---
    folder = os.path.basename(BASE_DIR.rstrip(os.sep))
    database_path = os.path.join(BASE_DIR, f"{folder} data.csv")
    with open(database_path, 'w', encoding='utf-8') as f:
        f.write(f"Model,,,,,,,,,,Dimensionless,,,,,,,,Prototype,{N_prototype/(2*np.pi/60)}, RPM\nMass Flow Rate [Kg/s],Total Pressure Rise [Bar],Torque [N-m],Shaft Power [Watts],Efficiency [%],Error Mass Flow Rate [Kg/s],Error Total Pressure Rise [Bar],Error Torque [N-m],Error Shaft Power [Watts],Error Efficiency [%],Pi Mass Flow Rate,Pi Total Pressure Rise,Pi Shaft Power,Pi Reynolds,Error Pi Mass Flow Rate,Error Pi Total Pressure Rise,Error Pi Shaft Power,Error Pi Reynolds,Prototype Mass Flow Rate [Kg/s],Prototype Total Pressure Rise [Bar], Prototype Shaft Power [Watts],Error Prototype Mass Flow Rate [Kg/s],Error Prototype Total Pressure Rise [Bar],Error Prototype Shaft Power [Watts]\n")
        
        for i in data_write_array:
            f.write(i)
    
    print(f"Database written to {database_path}")
    


=== Processing folder: /Users/krai/Library/CloudStorage/OneDrive-ChulalongkornUniversity/Electric Pump Senior Project/Experimental Record/Glycerine/7760 rpm/Glycerine_08%_7760_0.094Kg-s ===

=== Processing folder: /Users/krai/Library/CloudStorage/OneDrive-ChulalongkornUniversity/Electric Pump Senior Project/Experimental Record/Glycerine/7760 rpm/Glycerine_13%_7760_0.187Kg-s ===

=== Processing folder: /Users/krai/Library/CloudStorage/OneDrive-ChulalongkornUniversity/Electric Pump Senior Project/Experimental Record/Glycerine/7760 rpm/Glycerine_19%_7760_0.268Kg-s ===

=== Processing folder: /Users/krai/Library/CloudStorage/OneDrive-ChulalongkornUniversity/Electric Pump Senior Project/Experimental Record/Glycerine/7760 rpm/Glycerine_21%_7760_0.313Kg-s ===

=== Processing folder: /Users/krai/Library/CloudStorage/OneDrive-ChulalongkornUniversity/Electric Pump Senior Project/Experimental Record/Glycerine/7760 rpm/Glycerine_27.5%_7760_0.354Kg-s ===

=== Processing folder: /Users/krai/Library

In [42]:
#MACBOOK
#6730 rpm
BASE_DIR = r'/Users/krai/Library/CloudStorage/OneDrive-ChulalongkornUniversity/Electric Pump Senior Project/Experimental Record/Glycerine/6730 rpm'

subdirs = [
        r'Glycerine_08%_6730_0.081Kg-s',
        r'Glycerine_13%_6730_0.160Kg-s',
        r'Glycerine_19%_6730_0.235Kg-s',
        r'Glycerine_21%_6730_0.272Kg-s',
        r'Glycerine_25.6%_6730_0.302Kg-s',
        r'Glycerine_31.5%_6730_0.344Kg-s',
        r'Glycerine_36.5%_6730_0.378Kg-s',
        r'Glycerine_43%_6730_0.416Kg-s',
        r'Glycerine_52.5%_6730_0.454Kg-s',
        r'Glycerine_85%_6730_0.526Kg-s',
    ]

#7250 rpm
BASE_DIR = r'/Users/krai/Library/CloudStorage/OneDrive-ChulalongkornUniversity/Electric Pump Senior Project/Experimental Record/Glycerine/7250 rpm'

subdirs = [
        r'Glycerine_08%_7250_0.094Kg-s',
        r'Glycerine_13%_7250_0.175Kg-s',
        r'Glycerine_19%_7250_0.253Kg-s',
        r'Glycerine_21%_7250_0.290Kg-s',
        r'Glycerine_25.6%_7250_0.328Kg-s',
        r'Glycerine_31.5%_7250_0.370Kg-s',
        r'Glycerine_36.5%_7250_0.408Kg-s',
        r'Glycerine_43%_7250_0.447Kg-s',
        r'Glycerine_52.5%_7250_0.487Kg-s',
        r'Glycerine_85%_7250_0.565Kg-s',
    ]

#7760 rpm
BASE_DIR = r'/Users/krai/Library/CloudStorage/OneDrive-ChulalongkornUniversity/Electric Pump Senior Project/Experimental Record/Glycerine/7760 rpm'

subdirs = [
        r'Glycerine_08%_7760_0.094Kg-s',
        r'Glycerine_13%_7760_0.187Kg-s',
        r'Glycerine_19%_7760_0.268Kg-s',
        r'Glycerine_21%_7760_0.313Kg-s',
        r'Glycerine_27.5%_7760_0.354Kg-s',
        r'Glycerine_31.5%_7760_0.395Kg-s',
        r'Glycerine_36.5_7760_0.437Kg-s',
        r'Glycerine_43%_7760_0.477Kg-s',
        r'Glycerine_52.5%_7760_0.521Kg-s',
        r'Glycerine_85%_7760_0.603Kg-s',
    ]

#WINDOWS
#6730 rpm
BASE_DIR = r'C:\Users\Krai\OneDrive - Chulalongkorn University\Electric Pump Senior Project\Experimental Record\Glycerine\6730 rpm'

#7250 rpm
BASE_DIR = r'C:\Users\Krai\OneDrive - Chulalongkorn University\Electric Pump Senior Project\Experimental Record\Glycerine\7250 rpm'

#7760 rpm
BASE_DIR = r'C:\Users\Krai\OneDrive - Chulalongkorn University\Electric Pump Senior Project\Experimental Record\Glycerine\7760 rpm'


1.9749015599937718
